In [ ]:
# -*- coding: utf-8 -*-
try:
    import urllib.request as urllib2
except:
    import urllib2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import datetime
import folium
import mf2py
import json
from hcalendar import hCalendar

### a. Obtener y visualizar coordenadas geograficas

In [ ]:
def get_geo_data(name):
    """
        Devuelve los datos de geolocalizacion correspondiente
        a una ciudad registrada en Wikipedia.
    :param name: Nombre de la ciudad
    :return: Tupla con latitud y longitud
    """
    url = "https://es.wikipedia.org/wiki/" + name
    x = mf2py.Parser(url = url)
    x = x.to_dict(filter_by_type='h-card')[0]['properties']['geo'][0]
    x = x['value'].split(",")
    return x

In [ ]:
ciudades = ["Trelew", "Rawson", "Munich", "Nueva York", "Amsterdam"]

In [ ]:
coords = { ciudad : get_geo_data(ciudad) for ciudad in ciudades}

In [ ]:
mapa = folium.Map(location=[0, 0],zoom_start = 2)
for coord in coords:
    folium.Marker(coords[coord], popup=coord).add_to(mapa)
mapa.save('mapa.html')

In [ ]:
%%HTML
<iframe width="100%" height="350" src="mapa.html?inline=true"></iframe>

### b.Listar calendario de eventos.

In [ ]:
def get_calendar_data(name):
    """
        Devuelve los datos de fechas de salida de los episodios de series registradas en Wikipedia.
    :param name: Nombre de la serie
    :return: Coleccion de Tuplas con los nombres y fechas de emision
    """
    name = name.replace(" ", "_")
    url = "http://en.wikipedia.org/wiki/List_of_"+ name + "_episodes"
    file = urllib2.urlopen(urllib2.Request(url, headers={'User-agent': 'Python hCalendar'}))
    hcal = hCalendar(file)
    result = []
    for cal in hcal:
        for event in cal:
            result.append((event.summary, event.dtstart))
    return result

In [ ]:
series = ["House", "Friends", "Two and a Half Men", "The Big Bang Theory"]

In [ ]:
caps = { serie : get_calendar_data(serie) for serie in series}

In [ ]:
for c in caps:
    print('+'*90)
    print(c + ":")
    print('+'*90)
    for e in (caps[c]):
        print("Título: " + e[0] + "   Emisión: " + str(e[1].strftime("%d/%m/%Y")))

### b.Listar calendario de eventos - V2.

In [ ]:
def get_calendarEvent_data(name):
    """
        Devuelve los datos de eventos - deportes, conciertos, etc. - de diferentes ciudades
    :param name: Nombre de la ciudad
    :return: Coleccion de Tuplas con los eventos, el lugar y la fecha
    """
    name = name.replace(" ", "-")
    url = "http://www.superboleteria.com/boleto/eventos-" +name+ ".aspx"
    x = mf2py.Parser(url = url).to_dict(filter_by_type='h-event')
    result = []
    for e in x:
        name = e['properties']['name'][0]
        location = e['properties']['location'][0]['properties']['name'][0]
        start_time = e['properties']['start'][0]
        result.append((name, location, start_time))
    return result


In [ ]:
ciudades = ["California", "San Francisco", "Los Angeles"]

In [ ]:
eventos = { ciudad : get_calendarEvent_data(ciudad) for ciudad in ciudades}

In [ ]:
for e in eventos:
    print('+'*90)
    print(e + ":")
    print('+'*90)
    for i in (eventos[e]):
        print("*Evento: " + i[0] + "   Lugar: "+ i[1] + "   Fecha: "+ i[2])